In [2]:
if 1:
  !nvidia-smi

  !pip install -Uqq fastbook
  import fastbook
  fastbook.setup_book

  from google.colab import drive
  drive.mount('/content/drive')

Fri May  7 19:16:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from fastai.tabular.all import *

In [ ]:
creds = '{"username":"sandermann","key":"ca5b5b32a089bcc0a142517f68c433a6"}'

In [ ]:
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
  cred_path.parent.mkdir(exist_ok=True)
  cred_path.write_text(creds)
  cred_path.chmod(0o600)

In [4]:
from kaggle import *

OSError: ignored

In [ ]:
!kaggle competitions download -c tabular-playground-series-apr-2021

In [ ]:
path = URLs.path('playground_april')
path

In [ ]:
path.ls()

In [ ]:
file_extract(path/'tabular-playground-series-apr-2021.zip')

In [ ]:
df = pd.read_csv(path/'train.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
dep_var = 'Survived'

In [ ]:
procs = [Categorify, FillMissing, Normalize]

In [ ]:
cont, cat = cont_cat_split(df, dep_var=dep_var)

In [ ]:
idx = np.random.permutation(range(len(df))).squeeze()

In [ ]:
idx.shape

In [ ]:
to = TabularPandas(df, procs, cat, cont, dep_var, splits=[list(idx[:80000]), list(idx[80000:])])

In [ ]:
len(to.train), len(to.valid)

In [ ]:
to.show(3)

In [ ]:
dls = to.dataloaders(1024)

In [ ]:
y = to.train.y
y.min(), y.max()

In [ ]:
learn = tabular_learner(dls, [500,250], n_out=2, loss_func = CrossEntropyLossFlat(), metrics=accuracy)

In [ ]:
learn.lr_find(end_lr=1)

In [ ]:
learn.fit_one_cycle(5, 1e-3, wd=1e-3)

In [ ]:
learn.recorder.plot_loss()